# Specific Assets Data Grabber (Minute-Level Data)

This notebook fetches **minute-level** historical price and volume data for 15 specific cryptocurrencies:
- **Major:** BTC, ETH, BNB, SOL, ADA, AVAX, LINK
- **DeFi:** UNI, AAVE, MKR
- **Layer 2:** OP, ARB, STRK, POL/MATIC
- **Emerging:** NEAR

**Key Features:**
- **Minute-level granularity** (60-second candles)
- **Reverse chronological fetching** - starts from NOW and works backwards
- **Smart early stopping** - stops fetching when no more data is available for an asset
- Consolidated, non-redundant code
- Smart symbol resolution across multiple exchanges
- Comprehensive error handling and retry logic
- Final dataset statistics and quality report

In [11]:
import requests
import pandas as pd
import time
import os
from datetime import datetime, timedelta
from functools import reduce
from dotenv import load_dotenv

load_dotenv()

# Configuration
COINAPI_KEY = os.getenv('COIN_API_KEY')
GRANULARITY = 60  # 1 minute in seconds (changed from 3600)
TOTAL_MINUTES = 1051200  # total number of minutes to fetch (~2 years)
CHUNK_SIZE = 10000   # CoinAPI limit per request
MAX_RETRIES = 3
NO_DATA_THRESHOLD = 3  # Stop after this many consecutive empty responses

headers = {
    'Accept': 'application/json',
    'X-CoinAPI-Key': COINAPI_KEY
}

print(f"🔑 API Key loaded: {'✅ Yes' if COINAPI_KEY else '❌ No'}")
print(f"⏱️  Granularity: 1 minute")
print(f"📊 Target: {TOTAL_MINUTES:,} minutes (~{TOTAL_MINUTES/60/24/365:.1f} years)")

🔑 API Key loaded: ✅ Yes
⏱️  Granularity: 1 minute
📊 Target: 1,051,200 minutes (~2.0 years)


In [12]:
# Asset symbol configuration (consolidated from all testing)
# Using the working symbols discovered through iterative testing
SPECIFIC_ASSETS = {
    # Coinbase USD pairs (primary source)
    'BTC': 'COINBASE_SPOT_BTC_USD',
    'ETH': 'COINBASE_SPOT_ETH_USD', 
    'SOL': 'COINBASE_SPOT_SOL_USD',
    'ADA': 'COINBASE_SPOT_ADA_USD',
    'AVAX': 'COINBASE_SPOT_AVAX_USD',
    'LINK': 'COINBASE_SPOT_LINK_USD',
    'UNI': 'COINBASE_SPOT_UNI_USD',
    'AAVE': 'COINBASE_SPOT_AAVE_USD',
    'ARB': 'COINBASE_SPOT_ARB_USD',
    'MKR': 'COINBASE_SPOT_MKR_USD',  # Alternative that worked
    
    # Kraken USD pairs (backup exchange)
    'NEAR': 'KRAKEN_SPOT_NEAR_USD',
    'BNB': 'KRAKEN_SPOT_BNB_USD',
    'STRK': 'KRAKEN_SPOT_STRK_USD',
    'POL': 'KRAKEN_SPOT_POL_USD',    # Alternative that worked
    
    # Bitstamp (alternative exchange)
    'OP': 'BITSTAMP_SPOT_OP_USD'     # Alternative that worked
}

print(f"📋 Configured {len(SPECIFIC_ASSETS)} assets:")
for asset, symbol in SPECIFIC_ASSETS.items():
    exchange = symbol.split('_')[0]
    print(f"  {asset:6s}: {exchange}")

📋 Configured 15 assets:
  BTC   : COINBASE
  ETH   : COINBASE
  SOL   : COINBASE
  ADA   : COINBASE
  AVAX  : COINBASE
  LINK  : COINBASE
  UNI   : COINBASE
  AAVE  : COINBASE
  ARB   : COINBASE
  MKR   : COINBASE
  NEAR  : KRAKEN
  BNB   : KRAKEN
  STRK  : KRAKEN
  POL   : KRAKEN
  OP    : BITSTAMP


In [13]:
def get_historic_candles(name, symbol_id, granularity, total_minutes, chunk_size, now):
    """
    Fetch historical OHLCV data from CoinAPI, starting from now and going backwards.
    Stops early if consecutive empty responses are received.
    """
    all_data = []
    empty_response_count = 0
    
    for i in range(0, total_minutes, chunk_size):
        # Start from now and go backwards
        end_time = (now - timedelta(seconds=granularity * i)).replace(microsecond=0)
        start_time = (end_time - timedelta(seconds=granularity * chunk_size)).replace(microsecond=0)

        url = f"https://rest.coinapi.io/v1/ohlcv/{symbol_id}/history"
        params = {
            'period_id': '1MIN',  # Changed from 1HRS to 1MIN
            'time_start': start_time.isoformat() + 'Z',
            'time_end': end_time.isoformat() + 'Z',
            'limit': chunk_size
        }

        # Retry logic for rate limits
        for attempt in range(3):
            try:
                r = requests.get(url, headers=headers, params=params, timeout=30)
                if r.status_code == 429:
                    print(f"⏳ Rate limit hit for {name}. Waiting 60 seconds...")
                    time.sleep(60)
                    continue
                r.raise_for_status()
                data = r.json()
                
                if len(data) == 0:
                    empty_response_count += 1
                    print(f"⚠️  No data for {name} from {start_time} to {end_time} (empty count: {empty_response_count}/{NO_DATA_THRESHOLD})")
                    
                    # Stop after consecutive empty responses
                    if empty_response_count >= NO_DATA_THRESHOLD:
                        print(f"🛑 Stopping {name} after {NO_DATA_THRESHOLD} consecutive empty responses")
                        return pd.DataFrame(all_data)
                    break
                else:
                    # Reset counter when we get data
                    empty_response_count = 0
                
                all_data.extend(data)
                print(f"✅ Fetched {len(data)} rows from {name} | {start_time.strftime('%Y-%m-%d %H:%M')} to {end_time.strftime('%Y-%m-%d %H:%M')} | Total: {len(all_data):,}")
                break
                
            except Exception as e:
                if attempt == 2:  # Last attempt
                    raise Exception(f"Failed after 3 attempts: {e}")
                print(f"⚠️  Attempt {attempt + 1} failed: {e}. Retrying...")
                time.sleep(5)

        time.sleep(0.5)  # Polite delay between chunks

    # Process and clean data
    if not all_data:
        return pd.DataFrame()
        
    df = pd.DataFrame(all_data)
    df = df.rename(columns={
        'time_period_start': 'time',
        'price_close': f'{name}_close',
        'volume_traded': f'{name}_volume'
    })
    df['time'] = pd.to_datetime(df['time'])
    df = df[['time', f'{name}_close', f'{name}_volume']]
    df = df.sort_values('time').reset_index(drop=True)
    
    print(f"🎯 {name} complete: {len(df):,} total rows")
    return df


def fetch_asset_with_retry(name, symbol_id, max_retries=3):
    """Fetch data for an asset with comprehensive retry and error handling"""
    for attempt in range(1, max_retries + 1):
        try:
            print(f"🔄 Attempt {attempt}/{max_retries} for {name} ({symbol_id})...")
            
            # Start from current time (now) and work backwards
            df = get_historic_candles(
                name, symbol_id, GRANULARITY, TOTAL_MINUTES, CHUNK_SIZE, 
                datetime.utcnow().replace(second=0, microsecond=0)  # Start at current minute
            )
            
            if df is not None and not df.empty:
                df.to_csv(f"./specific_asset_data/{name}_data.csv", index=False)
                print(f"✅ Success! Saved {len(df)} rows for {name}")
                return True, None
            else:
                error_msg = "No data returned from API"
                
        except Exception as e:
            error_msg = str(e)
            print(f"⚠️ Attempt {attempt} failed: {error_msg}")
            
            # Specific error handling
            if "403" in error_msg or "Forbidden" in error_msg:
                print(f"   🚫 403 Forbidden - symbol may not exist on this exchange")
            elif "404" in error_msg:
                print(f"   🚫 404 Not Found - {symbol_id} may not exist")
            elif "422" in error_msg:
                print(f"   🚫 422 Unprocessable - symbol format issue")
        
        if attempt < max_retries:
            print(f"   Retrying in 5 seconds...")
            time.sleep(5)
    
    return False, error_msg


print("✅ Core functions defined")

✅ Core functions defined


In [14]:
def prep_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Prepare dataframe for merging: rename columns, parse time, handle duplicates"""
    df = df.copy()
    
    # Rename *_close -> *_price for consistency
    rename_map = {c: f"{c.rsplit('_',1)[0]}_price" for c in df.columns if c.endswith('_close')}
    if rename_map:
        df = df.rename(columns=rename_map)

    # Parse time and set as index
    df['time'] = pd.to_datetime(df['time'], format='ISO8601', utc=True)
    df = df.drop_duplicates('time').sort_values('time').set_index('time')

    # Handle bad price data
    for c in df.columns:
        if c.endswith('_price'):
            df[c] = df[c].mask(df[c] <= 0)  # Remove zero/negative prices
            
    return df


def merge_asof_many(dfs, tolerance='5min', direction='nearest'):
    """Merge multiple dataframes using pandas merge_asof for time-series alignment"""
    dfs = [prep_dataframe(d) for d in dfs]
    
    # Use the densest dataframe as the base timeline
    left = max(dfs, key=lambda d: len(d))
    others = [d for d in dfs if d is not left]
    
    tol = pd.Timedelta(tolerance)
    
    def merge_two(left, right):
        # Handle column name collisions
        intersect = set(left.columns).intersection(right.columns)
        if intersect:
            right = right.rename(columns={c: f"{c}_r" for c in intersect})
            
        return pd.merge_asof(
            left.sort_index(), right.sort_index(),
            left_index=True, right_index=True,
            direction=direction, tolerance=tol
        )
    
    # Sequentially merge all dataframes
    merged = reduce(merge_two, others, left)
    return merged.reset_index().rename(columns={'index': 'time'})


print("✅ Data merging functions defined")

✅ Data merging functions defined


In [15]:
# Main execution: Fetch data for all assets
os.makedirs('./specific_asset_data', exist_ok=True)

# Check existing files to avoid re-downloading
existing_files = []
missing_assets = {}

print("📂 Checking existing data files...")
for name, symbol_id in SPECIFIC_ASSETS.items():
    csv_path = f"./specific_asset_data/{name}_data.csv"
    if os.path.exists(csv_path):
        try:
            df = pd.read_csv(csv_path)
            if len(df) > 0:
                existing_files.append(name)
                print(f"✅ {name}: Already exists ({len(df):,} rows)")
                continue
        except Exception:
            pass
    
    missing_assets[name] = symbol_id
    print(f"❌ {name}: Will fetch")

print(f"\n📊 Status: {len(existing_files)} exist, {len(missing_assets)} to fetch")

# Fetch missing assets
if missing_assets:
    successful_fetches = []
    failed_fetches = []
    
    for name, symbol_id in missing_assets.items():
        print(f"\n🎯 Fetching data for {name}...")
        success, error = fetch_asset_with_retry(name, symbol_id, MAX_RETRIES)
        
        if success:
            successful_fetches.append(name)
        else:
            failed_fetches.append((name, error))
            print(f"❌ Final failure for {name}: {error}")
        
        time.sleep(2)  # Polite delay
    
    print(f"\n📊 Fetch Results:")
    print(f"✅ Successful: {len(successful_fetches)} ({', '.join(successful_fetches)})")
    if failed_fetches:
        print(f"❌ Failed: {len(failed_fetches)}")
        for name, error in failed_fetches:
            print(f"   {name}: {error}")
else:
    print("🎉 All assets already exist!")

📂 Checking existing data files...
❌ BTC: Will fetch
❌ ETH: Will fetch
❌ SOL: Will fetch
❌ ADA: Will fetch
❌ AVAX: Will fetch
❌ LINK: Will fetch
❌ UNI: Will fetch
❌ AAVE: Will fetch
❌ ARB: Will fetch
❌ MKR: Will fetch
❌ NEAR: Will fetch
❌ BNB: Will fetch
❌ STRK: Will fetch
❌ POL: Will fetch
❌ OP: Will fetch

📊 Status: 0 exist, 15 to fetch

🎯 Fetching data for BTC...
🔄 Attempt 1/3 for BTC (COINBASE_SPOT_BTC_USD)...


/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_74503/1417586807.py:85: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow().replace(second=0, microsecond=0)  # Start at current minute


✅ Fetched 6933 rows from BTC | 2025-11-04 20:28 to 2025-11-11 19:08 | Total: 6,933
⚠️  Attempt 1 failed: 500 Server Error: Internal Server Error for url: https://rest.coinapi.io/v1/ohlcv/COINBASE_SPOT_BTC_USD/history?period_id=1MIN&time_start=2025-10-28T21%3A48%3A00Z&time_end=2025-11-04T20%3A28%3A00Z&limit=10000. Retrying...
✅ Fetched 9944 rows from BTC | 2025-10-28 21:48 to 2025-11-04 20:28 | Total: 16,877
✅ Fetched 9664 rows from BTC | 2025-10-21 23:08 to 2025-10-28 21:48 | Total: 26,541
✅ Fetched 9998 rows from BTC | 2025-10-15 00:28 to 2025-10-21 23:08 | Total: 36,539
✅ Fetched 9968 rows from BTC | 2025-10-08 01:48 to 2025-10-15 00:28 | Total: 46,507
✅ Fetched 9984 rows from BTC | 2025-10-01 03:08 to 2025-10-08 01:48 | Total: 56,491
✅ Fetched 10000 rows from BTC | 2025-09-24 04:28 to 2025-10-01 03:08 | Total: 66,491
✅ Fetched 9997 rows from BTC | 2025-09-17 05:48 to 2025-09-24 04:28 | Total: 76,488
✅ Fetched 9994 rows from BTC | 2025-09-10 07:08 to 2025-09-17 05:48 | Total: 86,482


/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_74503/1417586807.py:85: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow().replace(second=0, microsecond=0)  # Start at current minute


✅ Fetched 6930 rows from ETH | 2025-11-04 20:31 to 2025-11-11 19:11 | Total: 6,930
✅ Fetched 9944 rows from ETH | 2025-10-28 21:51 to 2025-11-04 20:31 | Total: 16,874
✅ Fetched 9661 rows from ETH | 2025-10-21 23:11 to 2025-10-28 21:51 | Total: 26,535
✅ Fetched 9999 rows from ETH | 2025-10-15 00:31 to 2025-10-21 23:11 | Total: 36,534
✅ Fetched 9967 rows from ETH | 2025-10-08 01:51 to 2025-10-15 00:31 | Total: 46,501
✅ Fetched 9985 rows from ETH | 2025-10-01 03:11 to 2025-10-08 01:51 | Total: 56,486
✅ Fetched 10000 rows from ETH | 2025-09-24 04:31 to 2025-10-01 03:11 | Total: 66,486
✅ Fetched 9993 rows from ETH | 2025-09-17 05:51 to 2025-09-24 04:31 | Total: 76,479
✅ Fetched 9995 rows from ETH | 2025-09-10 07:11 to 2025-09-17 05:51 | Total: 86,474
✅ Fetched 9880 rows from ETH | 2025-09-03 08:31 to 2025-09-10 07:11 | Total: 96,354
✅ Fetched 9988 rows from ETH | 2025-08-27 09:51 to 2025-09-03 08:31 | Total: 106,342
✅ Fetched 9999 rows from ETH | 2025-08-20 11:11 to 2025-08-27 09:51 | Total

...

✅ Fetched 5735 rows from MKR | 2025-11-04 20:50 to 2025-11-11 19:30 | Total: 5,735
✅ Fetched 8921 rows from MKR | 2025-10-28 22:10 to 2025-11-04 20:50 | Total: 14,656
✅ Fetched 7818 rows from MKR | 2025-10-21 23:30 to 2025-10-28 22:10 | Total: 22,474
✅ Fetched 9052 rows from MKR | 2025-10-15 00:50 to 2025-10-21 23:30 | Total: 31,526
✅ Fetched 9469 rows from MKR | 2025-10-08 02:10 to 2025-10-15 00:50 | Total: 40,995
✅ Fetched 9628 rows from MKR | 2025-10-01 03:30 to 2025-10-08 02:10 | Total: 50,623
✅ Fetched 9484 rows from MKR | 2025-09-24 04:50 to 2025-10-01 03:30 | Total: 60,107
✅ Fetched 8838 rows from MKR | 2025-09-17 06:10 to 2025-09-24 04:50 | Total: 68,945
✅ Fetched 9825 rows from MKR | 2025-09-10 07:30 to 2025-09-17 06:10 | Total: 78,770
✅ Fetched 9846 rows from MKR | 2025-09-03 08:50 to 2025-09-10 07:30 | Total: 88,616
✅ Fetched 9975 rows from MKR | 2025-08-27 10:10 to 2025-09-03 08:50 | Total: 98,591
✅ Fetched 9993 rows from MKR | 2025-08-20 11:30 to 2025-08-27 10:10 | Total: 

/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_74503/1417586807.py:85: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow().replace(second=0, microsecond=0)  # Start at current minute


✅ Fetched 9998 rows from NEAR | 2025-11-04 20:52 to 2025-11-11 19:32 | Total: 9,998
✅ Fetched 9609 rows from NEAR | 2025-10-28 22:12 to 2025-11-04 20:52 | Total: 19,607
✅ Fetched 9987 rows from NEAR | 2025-10-21 23:32 to 2025-10-28 22:12 | Total: 29,594
✅ Fetched 9993 rows from NEAR | 2025-10-15 00:52 to 2025-10-21 23:32 | Total: 39,587
✅ Fetched 9998 rows from NEAR | 2025-10-08 02:12 to 2025-10-15 00:52 | Total: 49,585
✅ Fetched 9989 rows from NEAR | 2025-10-01 03:32 to 2025-10-08 02:12 | Total: 59,574
✅ Fetched 9994 rows from NEAR | 2025-09-24 04:52 to 2025-10-01 03:32 | Total: 69,568
✅ Fetched 9999 rows from NEAR | 2025-09-17 06:12 to 2025-09-24 04:52 | Total: 79,567
✅ Fetched 9989 rows from NEAR | 2025-09-10 07:32 to 2025-09-17 06:12 | Total: 89,556
✅ Fetched 9869 rows from NEAR | 2025-09-03 08:52 to 2025-09-10 07:32 | Total: 99,425
✅ Fetched 9937 rows from NEAR | 2025-08-27 10:12 to 2025-09-03 08:52 | Total: 109,362
✅ Fetched 9998 rows from NEAR | 2025-08-20 11:32 to 2025-08-27 10

/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_74503/1417586807.py:85: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow().replace(second=0, microsecond=0)  # Start at current minute


✅ Fetched 9995 rows from BNB | 2025-11-04 20:55 to 2025-11-11 19:35 | Total: 9,995
✅ Fetched 9593 rows from BNB | 2025-10-28 22:15 to 2025-11-04 20:55 | Total: 19,588
✅ Fetched 9988 rows from BNB | 2025-10-21 23:35 to 2025-10-28 22:15 | Total: 29,576
✅ Fetched 9990 rows from BNB | 2025-10-15 00:55 to 2025-10-21 23:35 | Total: 39,566
✅ Fetched 10000 rows from BNB | 2025-10-08 02:15 to 2025-10-15 00:55 | Total: 49,566
✅ Fetched 9992 rows from BNB | 2025-10-01 03:35 to 2025-10-08 02:15 | Total: 59,558
✅ Fetched 9994 rows from BNB | 2025-09-24 04:55 to 2025-10-01 03:35 | Total: 69,552
✅ Fetched 9998 rows from BNB | 2025-09-17 06:15 to 2025-09-24 04:55 | Total: 79,550
✅ Fetched 9968 rows from BNB | 2025-09-10 07:35 to 2025-09-17 06:15 | Total: 89,518
✅ Fetched 9836 rows from BNB | 2025-09-03 08:55 to 2025-09-10 07:35 | Total: 99,354
✅ Fetched 9926 rows from BNB | 2025-08-27 10:15 to 2025-09-03 08:55 | Total: 109,280
✅ Fetched 9996 rows from BNB | 2025-08-20 11:35 to 2025-08-27 10:15 | Total

In [16]:
# Combine all CSV files into final dataset
print("\n🔄 Creating final combined dataset...")

csv_directory = "./specific_asset_data"
output_file = "specific_assets_dataset_minute_granularity.csv"

# Load all CSV files
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]
print(f"Found {len(csv_files)} CSV files")

dataframes = []
file_stats = {}

for csv_file in csv_files:
    file_path = os.path.join(csv_directory, csv_file)
    df = pd.read_csv(file_path)
    asset_name = csv_file.split('_')[0]
    
    # Store statistics
    file_stats[asset_name] = {
        'rows': len(df),
        'date_start': df['time'].min() if 'time' in df.columns else 'Unknown',
        'date_end': df['time'].max() if 'time' in df.columns else 'Unknown'
    }
    
    # Handle different CSV formats
    if 'time_period_start' in df.columns:
        # CoinAPI format - rename columns
        df = df.rename(columns={
            "time_period_start": "time",
            "price_close": f"{asset_name}_close",
            "volume_traded": f"{asset_name}_volume"
        })
        df = df[["time", f"{asset_name}_close", f"{asset_name}_volume"]]
    
    dataframes.append(df)

# Merge all dataframes
print(f"🔄 Merging {len(dataframes)} DataFrames...")
merged = merge_asof_many(dataframes, tolerance='5min', direction='nearest')

# Save final dataset
merged.to_csv(output_file, index=False)
print(f"✅ Final dataset saved: {output_file}")
print(f"📊 Shape: {merged.shape}")


🔄 Creating final combined dataset...
Found 15 CSV files
🔄 Merging 15 DataFrames...
✅ Final dataset saved: specific_assets_dataset_minute_granularity.csv
📊 Shape: (1046523, 31)


In [17]:
# Comprehensive Final Statistics Report
print("="*80)
print("🎉 FINAL DATA COLLECTION REPORT")
print("="*80)

# Load final dataset for analysis
final_df = pd.read_csv(output_file)

# Dataset overview
print(f"\n📊 DATASET OVERVIEW:")
print(f"Total rows: {len(final_df):,}")
print(f"Total columns: {len(final_df.columns)}")
print(f"Date range: {final_df['time'].min()} to {final_df['time'].max()}")
time_span = (pd.to_datetime(final_df['time'].max()) - pd.to_datetime(final_df['time'].min())).days
print(f"Time span: {time_span:,} days (~{time_span/365:.1f} years)")

# Asset analysis
assets = [col.split('_')[0] for col in final_df.columns if '_price' in col]
print(f"\n🎯 ASSETS: {len(assets)}/{len(SPECIFIC_ASSETS)}")
print(f"Collected: {', '.join(sorted(assets))}")

# Coverage analysis
print(f"\n📈 DATA COVERAGE:")
print(f"{'Asset':<6} {'Rows':<8} {'Coverage':<10} {'Exchange':<10} {'Start':<12} {'End':<12}")
print("-" * 70)

coverage_stats = []
for asset in sorted(assets):
    price_col = f"{asset}_price"
    if price_col in final_df.columns:
        non_null = final_df[price_col].notna().sum()
        coverage_pct = (non_null / len(final_df)) * 100
        
        # Get exchange info
        symbol_id = SPECIFIC_ASSETS.get(asset, '')
        exchange = symbol_id.split('_')[0] if symbol_id else 'Unknown'
        
        # Date range for this asset
        asset_data = final_df[final_df[price_col].notna()]
        start_date = asset_data['time'].min()[:10] if not asset_data.empty else 'N/A'
        end_date = asset_data['time'].max()[:10] if not asset_data.empty else 'N/A'
        
        coverage_stats.append((asset, non_null, coverage_pct, exchange, start_date, end_date))
        print(f"{asset:<6} {non_null:<8,} {coverage_pct:<9.1f}% {exchange:<10} {start_date:<12} {end_date:<12}")

# Summary metrics
total_data_points = sum(stat[1] for stat in coverage_stats)
avg_coverage = sum(stat[2] for stat in coverage_stats) / len(coverage_stats)
best_asset = max(coverage_stats, key=lambda x: x[2])
worst_asset = min(coverage_stats, key=lambda x: x[2])

print(f"\n📊 SUMMARY:")
print(f"Total data points: {total_data_points:,}")
print(f"Average coverage: {avg_coverage:.1f}%")
print(f"Best coverage: {best_asset[0]} ({best_asset[2]:.1f}%)")
print(f"Worst coverage: {worst_asset[0]} ({worst_asset[2]:.1f}%)")

# Exchange distribution
exchange_counts = {}
for stat in coverage_stats:
    exchange = stat[3]
    exchange_counts[exchange] = exchange_counts.get(exchange, 0) + 1

print(f"\n🏦 EXCHANGES:")
for exchange, count in sorted(exchange_counts.items()):
    print(f"  {exchange}: {count} assets")

# Final metrics
file_size_mb = os.path.getsize(output_file) / (1024*1024)
success_rate = len(assets) / len(SPECIFIC_ASSETS) * 100

print(f"\n✅ SUCCESS METRICS:")
print(f"Assets requested: {len(SPECIFIC_ASSETS)}")
print(f"Assets collected: {len(assets)}")
print(f"Success rate: {success_rate:.1f}%")
print(f"Dataset size: {file_size_mb:.1f} MB")
print(f"\n🎯 DATASET READY FOR LSTM TRAINING!")
print(f"File: {output_file}")
print("="*80)

🎉 FINAL DATA COLLECTION REPORT

📊 DATASET OVERVIEW:
Total rows: 1,046,523
Total columns: 31
Date range: 2023-11-06 16:28:00+00:00 to 2025-11-10 16:24:00+00:00
Time span: 734 days (~2.0 years)

🎯 ASSETS: 15/15
Collected: AAVE, ADA, ARB, AVAX, BNB, BTC, ETH, LINK, MKR, NEAR, OP, POL, SOL, STRK, UNI

📈 DATA COVERAGE:
Asset  Rows     Coverage   Exchange   Start        End         
----------------------------------------------------------------------
AAVE   1,046,004 100.0    % COINBASE   2023-11-06   2025-11-10  
ADA    1,046,499 100.0    % COINBASE   2023-11-06   2025-11-10  
ARB    1,046,465 100.0    % COINBASE   2023-11-06   2025-11-10  
AVAX   1,046,488 100.0    % COINBASE   2023-11-06   2025-11-10  
BNB    271,478  25.9     % KRAKEN     2025-04-22   2025-11-10  
BTC    1,046,523 100.0    % COINBASE   2023-11-06   2025-11-10  
ETH    1,046,494 100.0    % COINBASE   2023-11-06   2025-11-10  
LINK   1,046,478 100.0    % COINBASE   2023-11-06   2025-11-10  
MKR    1,045,319 99.9     % CO